In [1]:
import os
os.environ['MOSTLY_PASSWORD'] = 'Mostly01!'

from mostlyai import MostlyAI
client = MostlyAI(
    # api_key = 'xxx',         # or MOSTLYAI_API_KEY env var; this is bound to a single user account
    # base_url = 'https://..'  # optional; defaults to our free version
)

In [13]:
# for g in client.generators.list():
#     print(g.id, g.name)

In [14]:
# cid = next(client.connectors.list()).id
# client.connectors.get('920a7f09-9713-4fb6-bcf7-7a1684fe5020')
# gid = next(client.generators.list()).id
# client.generators.get(gid)
# sid = next(client.synthetic_datasets.list()).id
# client.synthetic_datasets.get(sid)

In [2]:
g = client.generators.get('4b6f2725-2743-4da5-b0e9-60d9440e2e92')
# acc = g.tables[0].model.metrics.accuracy.univariate
no_of_columns = len(g.tables[0].columns)

In [3]:
g.model_dump()

{'id': UUID('4b6f2725-2743-4da5-b0e9-60d9440e2e92'),
 'name': 'Michi Test',
 'description': None,
 'training_status': <ProgressStatus.new: 'NEW'>,
 'metadata': {'created_at': datetime.datetime(2024, 2, 14, 13, 56, 22, 974622, tzinfo=TzInfo(UTC)),
  'owner_id': '95296f9e-0d12-4731-a981-5b3950ec19c3',
  'owner_name': 'Super Admin'},
 'accuracy': None,
 'usage': {'total_datapoints': None},
 'tables': [{'id': UUID('8b93c10f-c3bc-4fc3-9bed-40808a9ead27'),
   'source_connector': {'id': UUID('2f115805-4b86-45b3-b31c-d4e0d053b331'),
    'name': '2f115805-4b86-45b3-b31c-d4e0d053b331',
    'uri': '/connectors/2f115805-4b86-45b3-b31c-d4e0d053b331'},
   'location': 'public.account',
   'name': 'account',
   'primary_key': 'account_id',
   'columns': [{'id': UUID('feb1058d-8830-4d35-8147-764c0e71c844'),
     'name': 'account_id',
     'included': True,
     'model_encoding_type': <ModelEncodingType.categorical: 'CATEGORICAL'>},
    {'id': UUID('e3583610-72e2-417b-9af9-ce88dca0ae6b'),
     'name': '

In [2]:
g = client.generators.get("53f2b0fc-cbeb-4ff4-aefd-02ff0df0826b")

In [4]:
conf = client.generators.config("53f2b0fc-cbeb-4ff4-aefd-02ff0df0826b")

In [7]:
client.generators.create(**conf)

  0%|          | 0/6 [00:05<?, ?it/s]


Generator(client=<mostlyai.generators._MostlyGeneratorsClient object at 0x12031e560>, extra_key_values=None, id=UUID('5369e47a-b233-46b0-b990-c575e9b14ef5'), name='client', description=None, training_status=<ProgressStatus.new: 'NEW'>, metadata=Metadata(client=None, extra_key_values=None, created_at=datetime.datetime(2024, 2, 14, 20, 1, 54, 207234, tzinfo=TzInfo(UTC)), owner_id='95296f9e-0d12-4731-a981-5b3950ec19c3', owner_name='Super Admin'), accuracy=None, usage=GeneratorUsage(client=None, extra_key_values=None, total_datapoints=None), tables=[SourceTable(client=None, extra_key_values=None, id=UUID('dfdd7fa7-0f2c-4278-b745-4cb486076eea'), source_connector=BaseResource(client=None, extra_key_values=None, id=UUID('b658074c-1e83-4d98-8b42-bbcaa64ae74c'), name='b658074c-1e83-4d98-8b42-bbcaa64ae74c', uri='/connectors/b658074c-1e83-4d98-8b42-bbcaa64ae74c'), location='public.client', name='client', primary_key='client_id', columns=[SourceColumn(client=None, extra_key_values=None, id=UUID('0

In [4]:
g.training.start()

In [6]:
print(g.training.progress())

KeyboardInterrupt: 

In [3]:
sd = client.synthetic_datasets.create(generator_id=g.id)

TypeError: Object of type UUID is not JSON serializable

In [ ]:
g.config()
g.delete()
g.update(name='Michi Test')

In [ ]:
for g in client.generators.list():
    g.update(g.name + " hello")
    

In [15]:
# config = client.connectors.config('4cb8ed54-94e1-40b9-892e-66a36ef6fa6a')
# config["name"] = "Michi Test"
# del config["client"]
# client.connectors.create(**config)

In [3]:
config = client.connectors.config('4cb8ed54-94e1-40b9-892e-66a36ef6fa6a')

In [4]:
config

{'type': 'POSTGRES',
 'name': 'POSTGRES-13-Feb-2024T13:44:49Z',
 'accessType': 'SOURCE',
 'config': {'host': 'postgres-test.cppq7davohkj.eu-central-1.rds.amazonaws.com',
  'port': 5432,
  'username': 'postgres'},
 'secrets': {'password': '9372a5fd70e31262ae3e78a97eb60376a8488bea89d24397dda18322cd6d05fc'},
 'ssl': None,
 'testConnection': None}

In [5]:
client.connectors.create(**config)

Connector(client=<mostlyai.connectors._MostlyConnectorsClient object at 0x112ff3dc0>, extra_key_values=None, id=UUID('81b650dd-9884-4cc3-8d79-97b80d0a5617'), name='POSTGRES-13-Feb-2024T13:44:49Z', type=<ConnectorType.postgres: 'POSTGRES'>, access_type=<ConnectorAccessType.source: 'SOURCE'>, config={'host': 'postgres-test.cppq7davohkj.eu-central-1.rds.amazonaws.com', 'port': 5432, 'username': 'postgres'}, secrets={'password': '****************'}, ssl=None, metadata=Metadata(client=None, extra_key_values=None, created_at=datetime.datetime(2024, 2, 14, 18, 35, 31, 561448, tzinfo=TzInfo(UTC)), owner_id='95296f9e-0d12-4731-a981-5b3950ec19c3', owner_name='Super Admin'), table_id=None)

In [56]:
# config = client.generators.config('d69e8bb9-14b8-4397-9259-bb3a02c703ca')
# config["name"] = "Michi Test without columns"
# del config["client"]
# del config["tables"][0]["columns"]
# del config["tables"][0]["modelConfiguration"]
# config
g = client.generators.create(tables=[{
    "name": "hello",
    'sourceConnectorId': '2f115805-4b86-45b3-b31c-d4e0d053b331', 
    'location': 'public.account'
}])

In [57]:
g

Generator(client=<mostlyai.generators._MostlyGeneratorsClient object at 0x11ef1fbd0>, extra_key_values=None, id=UUID('0a14b266-977e-4c07-b5a7-473104e8c2b0'), name='account', description=None, training_status=<ProgressStatus.new: 'NEW'>, metadata=Metadata(client=None, extra_key_values=None, created_at=datetime.datetime(2024, 2, 14, 14, 28, 28, 761830, tzinfo=TzInfo(UTC)), owner_id='95296f9e-0d12-4731-a981-5b3950ec19c3', owner_name='Super Admin'), accuracy=None, usage=GeneratorUsage(client=None, extra_key_values=None, total_datapoints=None), tables=[SourceTable(client=None, extra_key_values=None, id=UUID('bd50dc56-7dde-4c54-94b0-2d2c5daa0c5e'), source_connector=BaseResource(client=None, extra_key_values=None, id=UUID('2f115805-4b86-45b3-b31c-d4e0d053b331'), name='2f115805-4b86-45b3-b31c-d4e0d053b331', uri='/connectors/2f115805-4b86-45b3-b31c-d4e0d053b331'), location='public.account', name='account', primary_key='account_id', columns=[SourceColumn(client=None, extra_key_values=None, id=UU

In [16]:
gid = '4b6f2725-2743-4da5-b0e9-60d9440e2e92'
client.generators.delete(gid)